As discussed at [here](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/210064) the dataset has inverted images.  
Let's check them.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd

In [ ]:
train_folder = "/kaggle/input/ranzcr-clip-catheter-line-classification/train/"
inverted_img_uid_l = [
    '1.2.826.0.1.3680043.8.498.93053605340693492468203536922883055634',
    '1.2.826.0.1.3680043.8.498.89369371707034087254309411362762932453',
    '1.2.826.0.1.3680043.8.498.40874263518848015471042617691509326469',
    '1.2.826.0.1.3680043.8.498.60784708544708592859086705347710043758',
    '1.2.826.0.1.3680043.8.498.85400081110981214468425786540292202327',
    '1.2.826.0.1.3680043.8.498.46805490620878014733000033680286522306',
    '1.2.826.0.1.3680043.8.498.12426606037326639593164801231383702795',
    '1.2.826.0.1.3680043.8.498.44981676356222715641792310487558318854',
    '1.2.826.0.1.3680043.8.498.92223955132523718945948760352349399544',
    '1.2.826.0.1.3680043.8.498.45667033584171921001890022815707001978',
    '1.2.826.0.1.3680043.8.498.62409687695240227890004324406035594441'
]

In [ ]:
len(inverted_img_uid_l)

In [ ]:
inverted_img_path_l = [train_folder+uid+".jpg" for uid in inverted_img_uid_l]

In [ ]:
fig = plt.figure()
for i,uid in enumerate(inverted_img_uid_l):
    ax = fig.add_subplot(2, 6, i+1)
    path = train_folder + uid + ".jpg"
    img = cv2.imread(path)
    assert(img is not None)
    ax.imshow(img)
plt.show()

Yeah, there are inverted. But are these the ALL of inverted images? I don't know...  
Let's do ML classification!

The number of known inverted images are small. So why don't we increase the inverted images by invert known normal images by ourself?

First. Pick up images which not inverted.

In [ ]:
train_path_l = glob.glob(train_folder + "*.jpg")
normal_path_l = [path for path in train_path_l if os.path.splitext(os.path.basename(path))[0] not in inverted_img_uid_l]
print(len(train_path_l))
print(len(normal_path_l))

In [ ]:
image_num = 100
target_img_path_l = normal_path_l[:image_num]

In [ ]:
plt_col_num = 10
fig = plt.figure()
for i,path in enumerate(target_img_path_l):
    ax = fig.add_subplot(len(target_img_path_l)/plt_col_num, plt_col_num, i+1)
    img = cv2.imread(path)
    assert(img is not None)
    ax.imshow(img)
plt.show()

All of above images looks not inverted.  
Next, pick up half of these images paths to invert. I'm going to invert when data loading time.

In [ ]:
import random
inverted_path_l = random.sample(target_img_path_l, len(target_img_path_l)//2)
normal_path_l = list(set(target_img_path_l) - set(inverted_path_l))

Make labels.

In [ ]:
df_label_normal = pd.DataFrame({"path": normal_path_l, "inverted":False})
df_label_inverted = pd.DataFrame({"path": inverted_path_l, "inverted":True})
df_label = pd.concat([df_label_normal, df_label_inverted])
df_label = df_label.sample(frac=1).reset_index(drop=True)
df_label.head()

Check the label balance.

In [ ]:
df_label["inverted"].value_counts()

# Now, ML time!

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import glob
import os
import json
# from skimage import io, transform
import cv2 as cv
import numpy as np
import re
import matplotlib.pyplot as plt
from torchvision import transforms

Create dataset.

In [ ]:
class MyDataset(Dataset):

    def __init__(self, label_df, img_size):
        self.label_df = label_df
        self.img_size = img_size

    def __len__(self):
        return self.label_df.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        row = self.label_df.loc[idx, :]
        
        img = cv.imread(row["path"])
        assert(img is not None)
        img = cv2.resize(img, (self.img_size, self.img_size))
        
        if row["inverted"]:
            img = cv2.bitwise_not(img)
        
        img = img.transpose(2, 0, 1).astype('float32')
        
        label_ans = torch.Tensor([0, 0])
        label_ans[int(row["inverted"])] = 1

        return img, row["inverted"]

Test dataset.

In [ ]:
dataset = MyDataset(df_label, 64)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=10, shuffle=True
)
image_l, label_ans = next(iter(dataloader))

In [ ]:
print(label_ans)

In [ ]:
plt_col_num = 10
fig = plt.figure()
for i,img in enumerate(image_l):
#     print(img.shape)
    ax = fig.add_subplot(2,5, i+1)
    img_np = img.to('cpu').detach().numpy().copy()
    img_np = img_np.transpose(1, 2, 0).astype("uint8")
    ax.imshow(img_np)
plt.show()

Define net.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.block1 = self.conv_block(c_in=3, c_out=64, dropout=0.1, kernel_size=5, stride=1, padding=2)
        self.block2 = self.conv_block(c_in=64, c_out=32, dropout=0.1, kernel_size=3, stride=1, padding=1)
        self.block3 = self.conv_block(c_in=32, c_out=32, dropout=0.1, kernel_size=3, stride=1, padding=1)
        self.lastcnn = nn.Conv2d(in_channels=32, out_channels=2, kernel_size=16, stride=1, padding=0)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.block1(x)
        x = self.maxpool(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.maxpool(x)
        x = self.lastcnn(x)
        return x

    def conv_block(self, c_in, c_out, dropout,  **kwargs):
        seq_block = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU(),
            nn.Dropout2d(p=dropout)
        )
        return seq_block

Prepare for learning.

In [ ]:
import torch.optim as optim

model = MyNet()
model = model.cuda()
input_img_size = 64

dataset = MyDataset(df_label, input_img_size)

train_length = int(0.7 * len(dataset))
test_length = len(dataset) - train_length
lengths = [train_length, test_length]


train_dataset, valid_dataset = torch.utils.data.random_split(
    dataset, lengths, generator=torch.Generator().manual_seed(42))

train_batch_num  = 8
train_loader = DataLoader(train_dataset, batch_size=train_batch_num, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4)

criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Learning.

In [ ]:
epochs = 20
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs = inputs.cuda()
        labels = labels.long().cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs).squeeze()
#         print(outputs, outputs.shape)
#         print(labels, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
    print(i, loss.item())

print('Finished Training')
torch.save(model, "./invert_detect_model.h5")

# Predict with the model.

In [ ]:
model = torch.load("./invert_detect_model.h5")
model = model.cuda()

In [ ]:
df_train = pd.DataFrame({"path": train_path_l, "inverted":False}) # Dummy for inverted column.
dataset = MyDataset(df_train, input_img_size)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=64, shuffle=False
)

In [ ]:
df_train.shape

In [ ]:
df_train_result = pd.DataFrame({"path": train_path_l, "false_score":np.nan, "true_score":np.nan})

In [ ]:
predicted_inverted_img_l = []
for i, (inputs, labels) in enumerate(dataloader, 0):
    inputs = inputs.cuda()
    outputs = model(inputs).squeeze()
    start_index = i * dataloader.batch_size
    df_train_result.loc[
        start_index: start_index+len(inputs)-1, ["false_score", "true_score"]
    ] = outputs.to('cpu').detach().numpy().copy()
    
    if i%10 == 0:
        print(f"{i}/{len(dataloader)} done")

In [ ]:
df_train_result_inverted = df_train_result[df_train_result["false_score"]<df_train_result["true_score"]]

In [ ]:
fig = plt.figure()
count = 0
for _,row in df_train_result_inverted.iterrows():
    count += 1
    ax = fig.add_subplot(6, 5, count)
    print(count, path)
    path = row["path"]
    img = cv2.imread(path)
    assert(img is not None)
    ax.imshow(img)
plt.show()

In [ ]:
print(len(inverted_img_path_l))
print(df_train_result_inverted["path"].isin(inverted_img_path_l).sum())

All of the image which mentioned by at [here](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/210064#1149491) found by prediction.  

Below images are not mentioned it the discussion.

In [ ]:
df_not_mentioned = df_train_result_inverted["path"][~df_train_result_inverted["path"].isin(inverted_img_path_l)]

plt_col_num = 10
fig = plt.figure()
i = 0
for path in df_not_mentioned:
    i += 1
    ax = fig.add_subplot(4, 5, i)
    print(i, path)
    img = cv2.imread(path)
    assert(img is not None)
    ax.imshow(img)
plt.show()


But several normal image were mis-labeled.  
Accidentary, some upside-down image found.

In [ ]:
up_side_down_image = [
    "/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.56964749951381900643748134536978560792.jpg",
    "/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.76129162274163220380041920805275862370.jpg",
    "/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.11582767971938057384592968535311883741.jpg",
    "/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.29092351703254040179552572484602410700.jpg",
    "/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.11620053814932996350746126485322079242.jpg",
]

In [ ]:
plt_col_num = 10
fig = plt.figure()
for i,path in enumerate(up_side_down_image):
    ax = fig.add_subplot(2, 3, i+1)
    img = cv2.imread(path)
    assert(img is not None)
    ax.imshow(img)
plt.show()